# Taller #24: Interfaces Multimodales: Uniendo Voz y Gestos
#### Desarrollado por: David Santiago Cruz Hernández

In [1]:
import speech_recognition as sr
import tkinter as tk
from pythonosc import udp_client
import threading
import cv2
import mediapipe as mp

# pip install -r requirements.txt


### Configuración del cliente OSC

In [2]:
IP_DESTINO = "127.0.0.1"  # IP del servidor (Processing)
PUERTO_OSC = 9124         # Puerto OSC
cliente_osc = udp_client.SimpleUDPClient(IP_DESTINO, PUERTO_OSC)
detener_hilo = threading.Event()

### Lista de comandos permitidos

In [3]:
COMANDOS_VOZ = {'cambiar', 'mover', 'mostrar'}
GESTOS_VALIDOS = {'mano_abierta', 'dos_dedos'}

### Estados actuales

In [4]:
estado_voz = None
estado_gesto = None

### Interfaz gráfica con Tkinter

In [5]:
class AppVisual(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Control por Voz y gestos")
        self.geometry("500x400")
        self.color_actual = "white"
        self.pos_x = 200
        self.label = tk.Label(self, text="Esperando comando...", bg=self.color_actual, width=30, height=10, font=("Arial", 20))
        self.label.place(x=self.pos_x, y=100)


    def actualizar_interfaz(self):
        global estado_voz, estado_gesto
        if estado_voz == "cambiar" and estado_gesto == "mano_abierta":
            self.color_actual = "blue" if self.color_actual == "white" else "red"
            self.label.config(bg=self.color_actual)
            cliente_osc.send_message("/accion", ["cambiar_color"])
            estado_voz = None
        elif estado_voz == "mover" and estado_gesto == "dos_dedos":
            self.pos_x += 20 if self.pos_x < 350 else -200
            self.label.place(x=self.pos_x, y=100)
            cliente_osc.send_message("/accion", ["mover_objeto"])
            estado_voz = None
        elif estado_voz == "mostrar":
            self.label.config(text="Mostrando...")
            cliente_osc.send_message("/accion", ["mostrar"])
            estado_voz = None
        self.after(100, self.actualizar_interfaz)

    def actualizar_estado_voz(self, texto):
        self.label.config(text=texto.capitalize())

### Función de reconocimiento de voz

In [6]:
def escuchar_voz(app):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        while True:
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=3)
                texto = r.recognize_google(audio, language="es-CO").lower()
                if texto in COMANDOS_VOZ:
                    print(f"[VOZ] Comando reconocido: {texto}")
                    global estado_voz
                    estado_voz = texto
                    # Programar actualización en el hilo principal de Tkinter
                    app.after(0, lambda: app.actualizar_estado_voz(texto))
            except sr.UnknownValueError:
                pass
            except Exception as e:
                print(f"[ERROR VOZ]: {e}")

### Función de detección de gestos

In [7]:
def detectar_gestos():
    global estado_gesto
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                if index_tip.y < thumb_tip.y:
                    estado_gesto = "mano_abierta"
                else:
                    estado_gesto = "cerrado"

                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                if index_tip.y < middle_tip.y:
                    estado_gesto = "dos_dedos"
        else:
            estado_gesto = None

        cv2.imshow('Gestos de Mano', frame)
        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

### Actualización periódica de la interfaz

### Iniciar hilos

In [ ]:
app = AppVisual()
threading.Thread(target=escuchar_voz, args=(app,), daemon=True).start()
threading.Thread(target=detectar_gestos, daemon=True).start()
app.after(100, app.actualizar_interfaz)
app.mainloop()

[VOZ] Comando reconocido: cambiar
[ERROR VOZ]: listening timed out while waiting for phrase to start
[ERROR VOZ]: listening timed out while waiting for phrase to start
